In [4]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import PandasTools

In [5]:
df = pd.read_excel( 'ci4001376_si_002.xlsx', header=0 )

In [6]:
df.head()

,COMPOUND_NAME,SMILES,CDK5_pIC50
0,2476896,CC(C)n1c(=O)ccc2c1nc(nc2)Nc3ccc(cc3)OC,8.008774
1,2476895,CCn1c(=O)ccc2c1nc(nc2)Nc3ccc(c(c3)OC)OC,7.522879
2,2476818,CCn1c(=O)ccc2c1nc(nc2)Nc3ccc(cc3)F,7.275724
3,2476898,CCC(CC)n1c(=O)ccc2c1nc(nc2)Nc3ccccc3,8.060481
4,2476893,CC(C)n1c(=O)ccc2c1nc(nc2)Nc3ccc(cc3)O,7.823909


In [62]:
PandasTools.AddMoleculeColumnToFrame( df, smilesCol="SMILES" )

In [63]:
def calc_fp_arr( mol, nBits=1024 ):
    arr =np.zeros((1,))
    fps = AllChem.GetMorganFingerprintAsBitVect( mol, 2, nBits=nBits  )
    DataStructs.ConvertToNumpyArray( fps, arr )
    return arr

In [160]:
X = np.array([ calc_fp_arr(mol) for mol in df.ROMol ])
Y = []
for pic50 in df.CDK5_pIC50:
    if pic50 > 7.0:
        Y.append(1.)
    else:
        Y.append(0.)
Y = np.array(Y)
print(sum(Y))

112.0


In [161]:
from sklearn.cross_validation import train_test_split

In [162]:
trainx, testx, trainy, testy = train_test_split(X, Y,  test_size=0.1 )


In [169]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from sklearn import metrics

In [164]:
#Simple DNN
model = Sequential()
model.add( Dense(500, input_dim=1024, activation='relu') )
model.add( Dense(10,  activation='relu') )
model.add( Dense(1, activation='sigmoid') )



In [165]:
model.compile( loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'] )

In [166]:
model.fit(trainx,trainy, nb_epoch=100, batch_size=200)

Epoch 1/100
207/207 [==============================] - 0s - loss: 0.6836 - acc: 0.6280     
Epoch 2/100
207/207 [==============================] - 0s - loss: 0.6270 - acc: 0.7391     
Epoch 3/100
207/207 [==============================] - 0s - loss: 0.5699 - acc: 0.7971     
Epoch 4/100
207/207 [==============================] - 0s - loss: 0.5282 - acc: 0.8213     
Epoch 5/100
207/207 [==============================] - 0s - loss: 0.4825 - acc: 0.8309     
Epoch 6/100
207/207 [==============================] - 0s - loss: 0.4479 - acc: 0.8309     
Epoch 7/100
207/207 [==============================] - 0s - loss: 0.4213 - acc: 0.8551     
Epoch 8/100
207/207 [==============================] - 0s - loss: 0.3949 - acc: 0.8406     
Epoch 9/100
207/207 [==============================] - 0s - loss: 0.3713 - acc: 0.8502     
Epoch 10/100
207/207 [==============================] - 0s - loss: 0.3522 - acc: 0.8696     
Epoch 11/100
207/207 [==============================] - 0s - loss: 0.3369 - acc

In [171]:
p = model.predict(testx)
ro = [ round(x[0]) for x in p]
metrics.classification_report(ro,testy)

'             precision    recall  f1-score   support\n\n        0.0       0.57      0.80      0.67        10\n        1.0       0.78      0.54      0.64        13\n\navg / total       0.69      0.65      0.65        23\n'

In [173]:
from keras.utils.visualize_util import model_to_dot, plot
from IPython.display import SVG


In [175]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Convolution2D, Convolution1D, MaxPooling1D, MaxPooling2D
from keras.optimizers import SGD

In [176]:
model2 = Sequential()

In [177]:
model2.add( Convolution1D( 1024, 3, border_mode='same', input_shape=(1,1024)  ) )

In [181]:
model.add(Convolution1D(1, 1, border_mode='same'))

Exception: Input 0 is incompatible with layer convolution1d_5: expected ndim=3, found ndim=2

In [182]:
from keras.layers import Embedding